In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
data = pd.read_csv('/content/Youtube01.csv')

In [ ]:
data.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [ ]:
data.drop(['COMMENT_ID', 'AUTHOR', 'DATE'], axis=1, inplace=True)

In [ ]:
data.head()

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [ ]:
data.count()

,0
CONTENT,1956
CLASS,1956


In [ ]:
data.isnull().sum()

,0
CONTENT,0
CLASS,0


In [ ]:
# identify max toekn
max_num = 0
for i in data['CONTENT']:
    token = i.split()
    if len(token) > max_num:
        max_num = len(token)
print(max_num)

213


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data["CONTENT"], data["CLASS"], test_size=0.2, random_state=42)


In [ ]:
print(X_train.shape)
print(y_train.shape)

(1564,)
(1564,)


In [ ]:
print(X_test.shape)
print(y_test.shape)

(392,)
(392,)


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize training and testing text
X_train_tokens = tokenizer(list(X_train), padding=True, truncation=True, max_length=max_num, return_tensors="pt")
X_test_tokens = tokenizer(list(X_test), padding=True, truncation=True, max_length=max_num, return_tensors="pt")

# Print shapes to verify
print("Training Data Shape:", len(X_train))
print("Testing Data Shape:", len(X_test))
print("Tokenized X_train Shape:", X_train_tokens["input_ids"].shape)
print("Tokenized X_test Shape:", X_test_tokens["input_ids"].shape)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Training Data Shape: 1564
Testing Data Shape: 392
Tokenized X_train Shape: torch.Size([1564, 213])
Tokenized X_test Shape: torch.Size([392, 213])


In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

# Convert labels to tensors (ensure they are in LongTensor format)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Custom PyTorch Dataset
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

# Create Dataset objects
train_dataset = TextDataset(X_train_tokens, y_train_tensor)
test_dataset = TextDataset(X_test_tokens, y_test_tensor)

# Evaluation function
def compute_metrics(pred):
    logits, labels = pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# Load Pretrained BERT Model for Classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # Binary classification

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="bert_classification_run",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none",  # <- This disables WandB logging
    no_cuda=False,
    load_best_model_at_end=True
)

# Trainer for fine-tuning BERT
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics  # Added the compute_metrics function
)

# Train the model
trainer.train()

# Run evaluation
results = trainer.evaluate()
print("Evaluation Results:", results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.146473,0.964286
2,No log,0.153940,0.974490
3,0.115900,0.177274,0.974490


Evaluation Results: {'eval_loss': 0.14647291600704193, 'eval_accuracy': 0.9642857142857143, 'eval_runtime': 4.6888, 'eval_samples_per_second': 83.603, 'eval_steps_per_second': 10.45, 'epoch': 3.0}


In [ ]:
# prompt: save the above model in a format i can use to load easily

# Save the trained model and tokenizer
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

# You can later load the model and tokenizer like this:
# from transformers import BertForSequenceClassification, BertTokenizer

# model = BertForSequenceClassification.from_pretrained("./saved_model")
# tokenizer = BertTokenizer.from_pretrained("./saved_model")


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')

In [ ]:
# prompt: zip and download./saved_model

!zip -r /content/saved_model.zip /content/saved_model
from google.colab import files
files.download("/content/saved_model.zip")


  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/config.json (deflated 49%)
  adding: content/saved_model/vocab.txt (deflated 53%)
  adding: content/saved_model/special_tokens_map.json (deflated 42%)
  adding: content/saved_model/tokenizer_config.json (deflated 75%)
  adding: content/saved_model/model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>